In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import sys
from keras.models import load_model
import os
import random
import math
import numpy as np
from imutils import paths

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
#import helpers
from Audio import Audio

In [3]:
audio = Audio(path='audio/save/')
trainlist=audio.generate_audio_list(p='train')

In [4]:
#valX,valY = audio.load_audio(p='val')
vallist = audio.generate_audio_list(p='val')

In [5]:
data,label = audio.load_audio('ABUSE','test')

[INFO] loading images...
(50,)


In [6]:
data.shape,label.toarray()

((50, 26, 121), array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]]))

In [7]:
labels=os.listdir('audio/save/')

In [8]:
model=load_model('audio_weights/first-improvement-11-0.82.hdf5')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [10]:
result=model.predict(data)

In [13]:
result=np.argmax(result,axis=1)

In [15]:
result,np.argmax(label.toarray(),axis=1)

(array([  2,   2,   2,   2, 238,   2,   2,   2,   2,   2,   5,   2,   2,
          2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
        322,   2,   2,   2, 436,   2,   2,   2,   2,  91,   2,   2,   2,
          2,   2,   2,   2,   2,   2,   2,   5,   2,   2, 322]),
 array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2]))

In [36]:
type(float(32)/float(50))

float

In [28]:
np.divide(sum(result==classes),len(classes))

0

In [40]:
pred={}
for i,label in enumerate(labels):
    data,classes = audio.load_audio(label,'test')
    result=model.predict(data)
    result=np.argmax(result,axis=1)
    classes=np.argmax(classes.toarray(),axis=1)
    #print(classes==result)
    acc=float(sum(result==classes))/float(len(classes))
    #print(type(acc),type(label))
    pred[label]=acc

[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] l

(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[INFO] loading images...
(50,)
[I

In [41]:
pred

{'ABOUT': 0.48,
 'ABSOLUTELY': 0.76,
 'ABUSE': 0.86,
 'ACCESS': 0.9,
 'ACCORDING': 0.86,
 'ACCUSED': 0.9,
 'ACROSS': 0.92,
 'ACTION': 0.8,
 'ACTUALLY': 0.8,
 'AFFAIRS': 0.88,
 'AFFECTED': 0.78,
 'AFRICA': 0.92,
 'AFTER': 0.76,
 'AFTERNOON': 0.92,
 'AGAIN': 0.72,
 'AGAINST': 0.7,
 'AGREE': 0.8,
 'AGREEMENT': 0.86,
 'AHEAD': 0.76,
 'ALLEGATIONS': 1.0,
 'ALLOW': 0.62,
 'ALLOWED': 0.58,
 'ALMOST': 0.68,
 'ALREADY': 0.82,
 'ALWAYS': 0.8,
 'AMERICA': 0.74,
 'AMERICAN': 0.76,
 'AMONG': 0.72,
 'AMOUNT': 0.6,
 'ANNOUNCED': 0.78,
 'ANOTHER': 0.76,
 'ANSWER': 0.7,
 'ANYTHING': 0.74,
 'AREAS': 0.84,
 'AROUND': 0.66,
 'ARRESTED': 0.96,
 'ASKED': 0.7,
 'ASKING': 0.94,
 'ATTACK': 0.92,
 'ATTACKS': 0.96,
 'AUTHORITIES': 0.94,
 'BANKS': 0.84,
 'BECAUSE': 0.52,
 'BECOME': 0.86,
 'BEFORE': 0.76,
 'BEHIND': 0.84,
 'BEING': 0.58,
 'BELIEVE': 0.74,
 'BENEFIT': 0.86,
 'BENEFITS': 0.78,
 'BETTER': 0.64,
 'BETWEEN': 0.78,
 'BIGGEST': 0.9,
 'BILLION': 0.72,
 'BLACK': 0.82,
 'BORDER': 0.74,
 'BRING': 0.8,
 'BRIT

In [46]:
sorted(pred.items(), key=lambda d:d[1], reverse = True) 

[('DECISION', 1.0),
 ('ALLEGATIONS', 1.0),
 ('EUROPEAN', 1.0),
 ('MAJOR', 1.0),
 ('OFFICIALS', 0.98),
 ('SUNSHINE', 0.98),
 ('ENERGY', 0.98),
 ('FOLLOWING', 0.98),
 ('IRELAND', 0.98),
 ('TEMPERATURES', 0.98),
 ('OPERATION', 0.98),
 ('POTENTIAL', 0.98),
 ('SIMPLY', 0.96),
 ('CHILDREN', 0.96),
 ('CAMPAIGN', 0.96),
 ('UNION', 0.96),
 ('LEVELS', 0.96),
 ('WESTMINSTER', 0.96),
 ('TOGETHER', 0.96),
 ('FINAL', 0.96),
 ('SPECIAL', 0.96),
 ('GREECE', 0.96),
 ('ATTACKS', 0.96),
 ('REFERENDUM', 0.96),
 ('SCOTLAND', 0.96),
 ('HUNDREDS', 0.96),
 ('INQUIRY', 0.96),
 ('GUILTY', 0.96),
 ('JUDGE', 0.96),
 ('CAMERON', 0.96),
 ('ARRESTED', 0.96),
 ('FRANCE', 0.94),
 ('ASKING', 0.94),
 ('SINGLE', 0.94),
 ('AUTHORITIES', 0.94),
 ('MINISTERS', 0.94),
 ('SIGNIFICANT', 0.94),
 ('TALKS', 0.94),
 ('EXACTLY', 0.94),
 ('WORKING', 0.94),
 ('CENTRAL', 0.94),
 ('MAJORITY', 0.94),
 ('EUROPE', 0.94),
 ('WELFARE', 0.94),
 ('CHIEF', 0.94),
 ('MILITARY', 0.94),
 ('SHORT', 0.94),
 ('BUDGET', 0.94),
 ('SCOTTISH', 0.94),
 (

In [47]:
sorted(pred.items(), key=lambda d:d[1], reverse = False) 

[('THERE', 0.34),
 ('ABOUT', 0.48),
 ('GOING', 0.48),
 ('THING', 0.5),
 ('BECAUSE', 0.52),
 ('THINK', 0.52),
 ('THINGS', 0.56),
 ('SPENT', 0.56),
 ('DOING', 0.58),
 ('OTHER', 0.58),
 ('QUESTION', 0.58),
 ('BEING', 0.58),
 ('ALLOWED', 0.58),
 ('THESE', 0.58),
 ('REPORT', 0.58),
 ('THEIR', 0.58),
 ('WHETHER', 0.58),
 ('HAPPEN', 0.6),
 ('AMOUNT', 0.6),
 ('HOUSE', 0.6),
 ('CRISIS', 0.6),
 ('GAMES', 0.6),
 ('LATER', 0.62),
 ('WHERE', 0.62),
 ('LEAVE', 0.62),
 ('ALLOW', 0.62),
 ('DEFICIT', 0.62),
 ('START', 0.62),
 ('THOUGHT', 0.64),
 ('REALLY', 0.64),
 ('BETTER', 0.64),
 ('LITTLE', 0.64),
 ('PHONE', 0.66),
 ('AROUND', 0.66),
 ('WITHOUT', 0.66),
 ('COULD', 0.66),
 ('NEVER', 0.66),
 ('THREE', 0.66),
 ('ORDER', 0.66),
 ('STAGE', 0.66),
 ('OFTEN', 0.66),
 ('PERSON', 0.68),
 ('MOVING', 0.68),
 ('PERSONAL', 0.68),
 ('WOULD', 0.68),
 ('ALMOST', 0.68),
 ('FOUND', 0.68),
 ('SPEND', 0.68),
 ('COURT', 0.68),
 ('LEVEL', 0.68),
 ('STAFF', 0.68),
 ('HAVING', 0.68),
 ('QUITE', 0.68),
 ('SAYING', 0.7),
 ('

In [6]:
from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,Conv3D,Conv1D
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D,MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import LSTM,Input,GlobalAveragePooling1D
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.layers.wrappers import TimeDistributed, Bidirectional
import tensorflow as tf
#from keras.layers.extra import TimeDistributedConvolution2D, TimeDistributedMaxPooling2D, TimeDistributedFlatten

In [7]:
class_num = 500

# mfcc

In [8]:
input_audio = Input(shape = (26,121))

conv1d_1 = Conv1D(64,kernel_size = 3,strides=1, padding = 'same',
               activation = 'relu')(input_audio)

batch_norm1 = BatchNormalization()(conv1d_1)

batch_norm1 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm1)

conv1d_2 = Conv1D(128,kernel_size = 3, strides = 1, padding = 'same',
               activation = 'relu')(batch_norm1)

batch_norm2 = BatchNormalization()(conv1d_2)
batch_norm2 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm2)

conv1d_2 = Conv1D(512,kernel_size = 3, strides = 1, padding = 'same',
               activation = 'relu')(batch_norm2)

batch_norm2 = BatchNormalization()(conv1d_2)
batch_norm2 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm2)

bilstm1 = Bidirectional(LSTM(256,return_sequences=True,name='bilstm1'))(batch_norm2)
#bilstm2 = Bidirectional(LSTM(256,return_sequences=True),name = 'bilstm2')(bilstm1)

avgpool = GlobalAveragePooling1D(name='audio_avg')(bilstm1)

dense1 = Dense(512,activation='relu')(avgpool)
batch_norm3 = BatchNormalization()(dense1)
dense2 = Dense(class_num,activation = 'softmax')(batch_norm3)
#maxpooling1d = MaxPooling1D(pool_size=2, padding='same')(batch_norm2)



Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [9]:
model = Model(inputs=[input_audio], 
              outputs=[dense2])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26, 121)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 26, 64)            23296     
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 128)           24704     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 128)           512       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 128)            0         
__________

In [10]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.003, decay=1e-6, momentum=0.9)
#adam = Adam(lr = 0.003,decay=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


In [11]:
type(trainlist)

list

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [13]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [14]:
print("[INFO] training network...")
#print(sample_number)
filepath="audio_weights/second-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
stopper = EarlyStopping(monitor='val_acc',patience=3)
callbacks_list = [checkpoint,stopper]

[INFO] training network...


In [15]:
sample_number = len(trainlist)
val_number = len(vallist)
batch = 64
history=model.fit_generator(audio.generate_arrays_from_file(filelist=trainlist),
                        validation_data=
                    audio.generate_arrays_from_file(filelist=vallist),
                        steps_per_epoch=sample_number // batch,
                        validation_steps = val_number//batch,
                        epochs=10, verbose=1,
                   callbacks = callbacks_list)

Epoch 1/10
7636/7636 [==============================] - 14560s 2s/step - loss: 1.8987 - acc: 0.5719 - val_loss: 1.0725 - val_acc: 0.7196

Epoch 00001: val_acc improved from -inf to 0.71955, saving model to audio_weights/second-improvement-01-0.72.hdf5
Epoch 2/10
7636/7636 [==============================] - 13416s 2s/step - loss: 0.7867 - acc: 0.7896 - val_loss: 0.8226 - val_acc: 0.7783

Epoch 00002: val_acc improved from 0.71955 to 0.77833, saving model to audio_weights/second-improvement-02-0.78.hdf5
Epoch 3/10
7636/7636 [==============================] - 13546s 2s/step - loss: 0.5523 - acc: 0.8495 - val_loss: 0.7443 - val_acc: 0.7986

Epoch 00003: val_acc improved from 0.77833 to 0.79856, saving model to audio_weights/second-improvement-03-0.80.hdf5
Epoch 4/10
7636/7636 [==============================] - 13583s 2s/step - loss: 0.4110 - acc: 0.8886 - val_loss: 0.7120 - val_acc: 0.8088

Epoch 00004: val_acc improved from 0.79856 to 0.80877, saving model to audio_weights/second-improvem

In [16]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.savefig('audioauc.png')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.savefig('audioloss.png')
plt.show()


NameError: name 'history' is not defined

In [8]:
testlist[0]

'audio/data/INCREASE/test/INCREASE_00030.wav'

In [9]:
#model.load_weights('weights-improvement-04-0.98.hdf5')
#model=load_model('audio_weights/first-improvement-10-0.82.hdf5')
model.evaluate_generator(audio.generate_arrays_from_file(testlist,batch_size=64),
                         steps=len(testlist)//64)

[0.793620569889362, 0.8155448717948718]

In [ ]:
model1 = Model(inputs=[input_audio], 
              outputs=[dense2])
model1.load_weights('audio_weights/first-improvement-06-0.95.hdf5')
model1.trainable=False
model1.summary()

In [49]:
model2 = Model(inputs=[model1.input],outputs=[model1.get_layer('inter').output])

In [50]:
model2.trainable=False

In [54]:
input_audio = Input(shape = (26,121))

In [59]:
intermediate = model2(input_audio)
bilstm1 = Bidirectional(LSTM(256,return_sequences=True))(intermediate)
bilstm2 = Bidirectional(LSTM(256,return_sequences=True),name = 'bilstm2')(bilstm1)

avgpool = Flatten()(bilstm2)

dense = Dense(512,activation='relu')(avgpool)
batch_norm3 = BatchNormalization()(dense)
output = Dense(class_num,activation = 'softmax')(batch_norm3)

In [60]:
xmodel=Model(inputs=[input_audio],outputs=[output])
#xmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 26, 121)           0         
_________________________________________________________________
model_10 (Model)             (None, 4, 256)            148352    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 4, 512)            1050624   
_________________________________________________________________
bilstm2 (Bidirectional)      (None, 4, 512)            1574912   
_________________________________________________________________
flatten_10 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               1049088   
_________________________________________________________________
batch_normalization_21 (Batc (None, 512)               2048      
__________

In [63]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.003, decay=1e-6, momentum=0.9)
#adam = Adam(lr = 0.003,decay=1e-6)
xmodel.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


In [64]:
print("[INFO] training network...")
#print(sample_number)
filepath="audio_weights/first-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
stopper = EarlyStopping(monitor='val_acc',patience=3)
callbacks_list = [checkpoint,stopper]

[INFO] training network...


In [65]:
sample_number = len(trainlist)
val_number = len(vallist)
batch = 64
xmodel.fit_generator(audio.generate_arrays_from_file(filelist=trainlist),
                        validation_data=
                    audio.generate_arrays_from_file(filelist=vallist),
                        steps_per_epoch=sample_number // batch,
                        validation_steps = val_number//batch,
                        epochs=15, verbose=1,
                   callbacks = callbacks_list)

Epoch 1/15
124/124 [==============================] - 155s 1s/step - loss: 0.2806 - acc: 0.9017 - val_loss: 0.2593 - val_acc: 0.9271

Epoch 00001: val_acc improved from -inf to 0.92708, saving model to audio_weights/first-improvement-01-0.93.hdf5
Epoch 2/15
124/124 [==============================] - 144s 1s/step - loss: 0.0592 - acc: 0.9864 - val_loss: 0.2426 - val_acc: 0.9323

Epoch 00002: val_acc improved from 0.92708 to 0.93229, saving model to audio_weights/first-improvement-02-0.93.hdf5
Epoch 3/15
124/124 [==============================] - 139s 1s/step - loss: 0.0271 - acc: 0.9979 - val_loss: 0.2404 - val_acc: 0.9349

Epoch 00003: val_acc improved from 0.93229 to 0.93490, saving model to audio_weights/first-improvement-03-0.93.hdf5
Epoch 4/15
124/124 [==============================] - 140s 1s/step - loss: 0.0156 - acc: 0.9997 - val_loss: 0.2444 - val_acc: 0.9375

Epoch 00004: val_acc improved from 0.93490 to 0.93750, saving model to audio_weights/first-improvement-04-0.94.hdf5
Epo

In [ ]:
# intermediate_layer_model = Model(inputs=xmodel.input,
#                                  outputs=xmodel.get_layer('bilstm2').output)
# newmodel.compile(loss="categorical_crossentropy", optimizer=sgd,
# 	metrics=["accuracy"])
intermediate = xmodel.get_layer('bilstm2').output
avgpool1 = GlobalAveragePooling1D()(intermediate)

dense11 = Dense(256,activation='relu')(avgpool1)
batch_norm11 = BatchNormalization()(dense11)
dense11 = Dense(64,activation='relu')(batch_norm11)
batch_norm12 = BatchNormalization()(dense11)
dense12 = Dense(class_num,activation = 'softmax')(batch_norm12)
#m = intermediate_layer_model.predict(testX)


In [ ]:
model1 = Model(inputs=[xmodel.input], 
              outputs=[dense12])
model1.summary()

In [ ]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9)
model1.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

In [ ]:
sample_number = len(trainlist)
model1.fit_generator(audio.generate_arrays_from_file(filelist=trainlist),
                        validation_data=(valX, val_Y),
                        steps_per_epoch=sample_number // 64,
                        epochs=5, verbose=1)

In [ ]:
model1.fit(trainX, train_Y, epochs=5, batch_size=64, 
          verbose=1, validation_data=(valX, val_Y))

In [ ]:
model.evaluate(testX,test_Y)